In [180]:
import numpy as np
import itertools as iter

In [181]:
def create_G(r:int, m:int) -> np.ndarray[np.ndarray[int]]:
    n = 2**m
    E = [2**i for i in range(m-1, -1, -1)]
    arr = [0]
    for i in range(1, r+1):
        arr += [sum(j) for j in list(iter.combinations(E, i))]

    G = np.zeros((len(arr), n), dtype=int)
    for i in range(len(arr)):
        for j in range(n-1, -1, -1):
            if arr[i] & j == arr[i]:
                G[i][n-j-1] = 1
    return G

In [182]:
def find_special(arr: np.ndarray[int], n:int):
    """Находим unit_row, steps, block для v(Jc)"""
    unit_row = n
    for j in range(n):
        if arr[j] == 0:
            unit_row = j
            break
    j = unit_row
    steps = 0
    for j in range(unit_row, n, unit_row):
        if arr[j] == 0:
            steps += 1
        else:
            break
    x = unit_row*(steps+1)
    block = n
    for j in range(x, n, x):
        if arr[j] == 0:
            block = j
            break
    return unit_row, steps+1, block

In [183]:
def create_decode_array(r: int, m: int):
    """
    Создаём набор из матриц v(jc,t)
    Нам нужно вычислить v(Jc, t). v(Jc) вычисляется, как в G.
    Можно заметить, что остальные строки в v(Jc, t) - это сдвиг предыдущей на некоторые значения.

    unit_row - количество подряд идущих единиц в массиве
    steps - число возможных шагов сдвига на unit_row до следующих единиц

                        |  1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0  |
    Пример сдвига step: |  0 0 1 1 0 0 1 1 0 0 0 0 0 0 0 0  |

    block - величина блока, который будем сдвигать, если не можем делать сдвиги на unit_row

                         |  1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0  |
    Пример сдвига block: |  0 0 0 0 0 0 0 0 1 1 0 0 1 1 0 0  |


    Комбинируя сдвиги step и block получим v(Jc, t)

             |  1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0  |
    Пример:  |  0 0 1 1 0 0 1 1 0 0 0 0 0 0 0 0  | - step-сдвиг (всегда от предыдущей строки)
             |  0 0 0 0 0 0 0 0 1 1 0 0 1 1 0 0  | - block-сдвиг (всегда от i-(step+1) строки
             |  0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 1  | - step-сдвиг

    """
    n = 2**m
    E = [2 ** i for i in range(m - 1, -1, -1)]
    decode_array = []
    for k in range(r, 0, -1):  # k = |J|
        arr = []
        for x in [sum(j) for j in list(iter.combinations(E, m-k))]:
            v_t_array = np.zeros((2**(m-k), n), dtype=int)
            for j in range(n - 1, -1, -1):  # получаем v(Jc)
                if x & j == x:
                    v_t_array[0][n - j - 1] = 1
            unit_row, steps, block = find_special(v_t_array[0], n)
            for i in range(1, len(v_t_array)):
                if i % steps == 0:  # block-сдвиг
                    v_t_array[i, block:] = v_t_array[i-steps, :n-block]
                else:  # step-сдвиг
                    v_t_array[i, unit_row:] = v_t_array[i-1, :n-unit_row]
            arr.append(v_t_array.T)
        decode_array.append(np.array(arr))
    decode_array.append([np.eye(n, dtype=int)])
    return decode_array

In [184]:
def decoding_word(w, decode_array, G, r, m):
    """
    В decode_array лежат матрицы v(jc,t). Нам нужны ещё v(j,t).
    Можно заметить, что i-я v(j,t) соответствует i-ой строке в G
    """
    i = len(G) - 1  # переменная для пробега по G с конца
    word = np.zeros(i+1, dtype=int)  # декодированное слово
    w_ = w.copy()  # w(i) в лекции
    for v_t_array in decode_array:
        w_next = w_.copy()  # w(i-1) в лекции
        threshold_amount = len(v_t_array[0][0]) // 2  # 2**(m-i-1) в лекции
        for x in v_t_array:
            units_amount = (np.dot(w_, x) % 2).sum()
            if units_amount == threshold_amount:
                print(f'допущена {2**(m-r-1)}-я ошибка. Сообщение не может быть восстановлено!')
                return None
            if units_amount > threshold_amount:
                w_next ^= G[i]
                word[i] = 1
            else:
                word[i] = 0
            i -= 1
        w_ = w_next
    return word

**𝑅𝑀(2,4)**

In [185]:
R = 2
M = 4
G = create_G(R, M)
G

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0],
       [1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0],
       [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0],
       [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0]])

In [186]:
decode_array = create_decode_array(R, M)
decode_array

[array([[[1, 0, 0, 0],
         [1, 0, 0, 0],
         [1, 0, 0, 0],
         [1, 0, 0, 0],
         [0, 1, 0, 0],
         [0, 1, 0, 0],
         [0, 1, 0, 0],
         [0, 1, 0, 0],
         [0, 0, 1, 0],
         [0, 0, 1, 0],
         [0, 0, 1, 0],
         [0, 0, 1, 0],
         [0, 0, 0, 1],
         [0, 0, 0, 1],
         [0, 0, 0, 1],
         [0, 0, 0, 1]],
 
        [[1, 0, 0, 0],
         [1, 0, 0, 0],
         [0, 1, 0, 0],
         [0, 1, 0, 0],
         [1, 0, 0, 0],
         [1, 0, 0, 0],
         [0, 1, 0, 0],
         [0, 1, 0, 0],
         [0, 0, 1, 0],
         [0, 0, 1, 0],
         [0, 0, 0, 1],
         [0, 0, 0, 1],
         [0, 0, 1, 0],
         [0, 0, 1, 0],
         [0, 0, 0, 1],
         [0, 0, 0, 1]],
 
        [[1, 0, 0, 0],
         [0, 1, 0, 0],
         [1, 0, 0, 0],
         [0, 1, 0, 0],
         [1, 0, 0, 0],
         [0, 1, 0, 0],
         [1, 0, 0, 0],
         [0, 1, 0, 0],
         [0, 0, 1, 0],
         [0, 0, 0, 1],
         [0, 0, 1, 0],
     

In [187]:
word = np.array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
w = np.dot(word, G) % 2
w

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [188]:
err1_array = [w ^ x for x in np.eye(len(w), dtype=int)]  # все одномерные ошибки
err2 = np.zeros_like(w)
err2[0] = err2[1] = 1
err3 = err2.copy()
err3[2] = 1
w_err2 = w ^ err2
w_err3 = w ^ err3
w, w_err2, w_err3

(array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 array([0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 array([0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]))

Декодирование слова без ошибки

In [189]:
decode_word = decoding_word(w, decode_array, G, R, M)
if decode_word is not None:
    print(decode_word)
    print(np.array_equal(word, decode_word))

[1 0 0 0 0 0 0 0 0 0 0]
True


Декодирование всех однократных ошибок

In [190]:
decode_words = [decoding_word(x, decode_array, G, R, M) for x in err1_array]
for x in decode_words:
    print(np.array_equal(x, word))

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


Декодирование двукратной ошибки (RM(2, 4) может их лишь обнаружить, но не исправить)

In [191]:
decode_word = decoding_word(w_err2, decode_array, G, R, M)
if decode_word is not None:
    print(decode_word)
    print(np.array_equal(word, decode_word))

допущена 2-я ошибка. Сообщение не может быть восстановлено!


Декодирование 3-кратной ошибки (RM(2,4) не может их обнаружить)

In [192]:
decode_word = decoding_word(w_err3, decode_array, G, R, M)
if decode_word is not None:
    print(decode_word)
    print(np.array_equal(word, decode_word))

[1 0 0 0 0 1 0 0 0 0 0]
False
